# RIDSANS Mantid command interface

In [1]:
from reduction_workflow.command_interface import ReductionSingleton, Clear
from mantid.api import *
from mantid.kernel import *
from sansdata import *

## 1. Define and register a new reduction algorithms

Here, the two `PythonAlgorithm

In [2]:
# Derivative of HFIRSANSReduction

class SetupRIDSANSReduction(PythonAlgorithm):
    def category(self):
        return "Workflow\\SANS\\UsesPropertyManager"

    def name(self):
        return "SetupRIDSANSReduction"

    def summary(self):
        return "Setup for RID SANS reduction workflow."

    def PyInit(self):
        self.declareProperty("Filename", "", doc="List of input file paths")
        self.declareProperty(
            "ReductionProperties",
            "__sans_reduction_properties",
            validator=StringMandatoryValidator(),
            doc="Property manager name for the reduction",
        )
        self.declareProperty("OutputMessage", "", direction=Direction.Output, doc="Output message")

    def PyExec(self):
        print("SetupRIDSANSReduction execution (pass)")
        self.setPropertyValue("OutputMessage", "HFIR reduction options set");
        pass

AlgorithmFactory.subscribe(SetupRIDSANSReduction)

In [3]:
# Derivative of HFIRSANSReduction

class RIDSANSReduction(PythonAlgorithm):
    def category(self):
        return "Workflow\\SANS\\UsesPropertyManager"

    def name(self):
        return "RIDSANSReduction"

    def summary(self):
        return "RID SANS reduction workflow."

    def PyInit(self):
        self.declareProperty("Filename", "", doc="List of input file paths")
        self.declareProperty(
            "ReductionProperties",
            "__sans_reduction_properties",
            validator=StringMandatoryValidator(),
            doc="Property manager name for the reduction",
        )
        self.declareProperty("OutputWorkspace", "", doc="Reduced workspace")
        self.declareProperty("OutputMessage", "", direction=Direction.Output, doc="Output message")

    def PyExec(self):
        pass
    
AlgorithmFactory.subscribe(RIDSANSReduction)

In [ ]:
# Verify that SetupRIDSANSReduction, RIDSANSReduction algorithsm can now be created
setup_alg = AlgorithmManager.create("SetupRIDSANSReduction")
alg = AlgorithmManager.create("RIDSANSReduction")
setup_alg.initialize()
alg.initialize()
print(setup_alg,alg)

## 2. Define a new load algorithm for NXSas

In [ ]:
from mantid.simpleapi import *
from mantid.api import MatrixWorkspace

sans_file = SansData("data/memb_BS_Q1_6_0Ang.mpa")
sans_file.plot_2d()

In [ ]:
nBins = 1

delta_L_over_L0 = 0.1
bin_lower = sans_file.L0 * (1.0 - delta_L_over_L0/2)
bin_upper = sans_file.L0 * (1.0 + delta_L_over_L0/2)

detectors = 2

# Define single wavelength bin
x_single = np.array([bin_lower, bin_upper])
# Counts
y_single = np.array([0])

dataX = np.repeat(x_single,detectors)

dataY = np.repeat(y_single, detectors)

# The workspace will be named "dataWS1", error values will be zero.
dataWS1 = CreateWorkspace(OutputWorkspace = "first",UnitX="Wavelength",DataX=dataX, DataY=dataY, NSpec=detectors)

In [6]:
# Derivative of HFIRSANSReduction

class LoadRIDSANS(PythonAlgorithm):
    def category(self):
        return "Workflow\\SANS\\UsesPropertyManager"

    def name(self):
        return "LoadRIDSANS"

    def summary(self):
        return "RID SANS file loading."

    def PyInit(self):
        self.declareProperty("Filename", "", doc="List of input file paths")
        self.declareProperty("OutputWorkspace", "", doc="Loaded workspace")
        self.declareProperty("OutputMessage", "", direction=Direction.Output, doc="Output message")

    def PyExec(self):
        print("LoadRIDSANS execution (pass)")
        self.setPropertyValue("OutputMessage", "File loaded");
        file = self.getPropertyValue("Filename")
        samples = list(map(SansData, file))
        pass

AlgorithmFactory.subscribe(LoadRIDSANS)

## 3. Define RIDSANS command interface

In [7]:
def RIDSANS():
    Clear()
    ReductionSingleton().set_instrument("RID", "SetupRIDSANSReduction", "RIDSANSReduction")

In [57]:
RIDSANS()
# TODO: set all properties of the singleton by accessing ReductionSingleton().reduction_properties["PROPERTY_NAME"] 